## QuantAQ acquisition: South Bronx devices, November 7–10, 2024

This notebook focuses **only on data acquisition** from QuantAQ.

### What is QuantAQ?

QuantAQ devices are **air-quality monitors** that report time-series measurements (often including particulate matter and supporting environmental variables, depending on the device model and configuration). In this lesson, QuantAQ provides a **dense, neighborhood-scale monitoring network** in the South Bronx.

In the South Bronx, this dense QuantAQ network exists thanks to the environmental justice work of **South Bronx Unite** and research partners including **Dr. Markus Hilpert** and collaborators. The sample QuantAQ data used in this lesson was provided through this collaboration.

QuantAQ data are especially useful for:

- Seeing **neighborhood-scale patterns** when many devices are deployed in one area,
- Comparing **time patterns** across different data sources (data cleaning and alignment happen later),
- Supporting short “event window” analysis with **high-frequency measurements**.

As with any sensor dataset, these measurements should be treated as **raw observations** that may require quality checks before drawing strong conclusions. Those steps are discussed further in this lesson’s `m201-air-quality-measures-integrated` notebook.

### What this notebook does

- Authenticates with the QuantAQ API,
- Lists devices available to the provided API key (for this lesson, access is within the South Bronx Unite / Hilpert network),
- Downloads measurements for each device for the event window (Nov 7–10, 2024),
- Saves outputs to `data/raw/quantaq/` for later integration.

No plotting, resampling, or integrated analysis is performed here. Those steps will happen later in
the `m201-air-quality-measures-integrated` notebook.


## QuantAQ data access

QuantAQ data are accessed through an API using an API key.

- Create a QuantAQ account and generate an API key from the QuantAQ web application.
- This notebook will prompt you for the key using `getpass` so it is not printed to the screen.
- Note: access to the **South Bronx Unite / Hilpert** South Bronx network is not public; this lesson assumes your credentials have already been granted access.


## Install the QuantAQ Python client

If the `quantaq` package is not installed in your environment, install it from the official repository.


In [1]:
try:
    import quantaq
    print("quantaq is already installed")
except ImportError:
    !pip install git+https://github.com/quant-aq/py-quantaq.git

quantaq is already installed


## Set up the API client

Enter your QuantAQ API key when prompted. The key will be used only for this notebook session.


In [2]:
# import the library
import quantaq
import os, getpass

# Prompt for the API key (not echoed) and set it for this notebook session
if "QUANTAQ_APIKEY" not in os.environ or not os.environ["QUANTAQ_APIKEY"]:
    os.environ["QUANTAQ_APIKEY"] = getpass.getpass("Enter your QuantAQ API key: ")

# Initialize the QuantAQ client
client = quantaq.QuantAQAPIClient(api_key=os.environ["QUANTAQ_APIKEY"])


## Verify access

These quick checks confirm that your API key works and that you can see organizations and devices associated with your account.


In [3]:
# verify account information
whoami = client.whoami()
print (whoami)
# You can retrieve a list of all the organizations visible to you:
organizations = client.organizations.list()
print (organizations)
# You can retrieve a list of all the devices visible to you:
devices = client.devices.list()
for device in  devices:
    print(device["description"])

{'confirmed': True, 'email': 'topstschool@ciesin.columbia.edu', 'first_name': 'TOPS', 'id': 2704, 'is_administrator': False, 'last_name': 'SCHOOL', 'last_seen': '2025-08-19T17:49:01.880539', 'member_since': '2025-01-31T14:38:35.980542', 'role': 1, 'username': 'topstschool'}
[{'created_on': '2023-10-18T20:43:11.197345+00:00', 'description': 'South Bronx Unite brings together neighborhood residents, community organizations, academic institutions, and allies to improve and protect the social, environmental, and economic future of Mott Haven and Port Morris.', 'devices': ['MOD-PM-01325', 'MOD-PM-01163', 'MOD-PM-01317', 'MOD-PM-01158', 'MOD-PM-01153', 'MOD-PM-01147', 'MOD-PM-01323', 'MOD-PM-01328', 'MOD-PM-01463', 'MOD-PM-01334', 'MOD-PM-01330', 'MOD-00478', 'MOD-PM-01146', 'MOD-PM-01152', 'MOD-PM-01160', 'MOD-PM-01145', 'MOD-PM-01157', 'MOD-00693', 'MOD-00780', 'MOD-PM-01154', 'MOD-PM-01324', 'MOD-00695', 'MOD-PM-01161', 'MOD-00697', 'MOD-00480', 'MOD-PM-01320', 'MOD-PM-01144', 'MOD-PM-011

## Optional: preview a device and a few recent records

This is a small “smoke test” that the API is returning data. It pulls a handful of recent records from a single device.


In [4]:
# You can limit the return to just the most recent data points from a single device:
recent = client.data.list(sn='MOD-00480', sort="timestamp,asc", limit=10)
for r in recent:
    print(r)

{'co': None, 'geo': {'lat': 40.806, 'lon': -73.93}, 'met': {'rh': 90.0, 'temp': 6.1, 'wd': 0.0, 'ws': None, 'ws_scalar': 0.0}, 'model': {'gas': {'co': None, 'no': None, 'no2': None, 'o3': None}, 'pm': {'pm1': 11669, 'pm10': 11671, 'pm25': 11670}}, 'no': None, 'no2': None, 'o3': None, 'pm1': 4.18, 'pm10': 167.16, 'pm25': 6.95, 'raw_data_id': 83357948, 'rh': 90.0, 'sn': 'MOD-00480', 'temp': 6.1, 'timestamp': '2000-01-01T00:01:01', 'timestamp_local': '1999-12-31T19:01:01', 'url': 'https://api.quant-aq.com/v1/devices/MOD-00480/data/83357866'}
{'co': None, 'geo': {'lat': None, 'lon': None}, 'met': {'rh': 39.2, 'temp': 25.9, 'wd': 0.0, 'ws': None, 'ws_scalar': 0.0}, 'model': {'gas': {'co': None, 'no': None, 'no2': None, 'o3': None}, 'pm': {'pm1': 11669, 'pm10': 11671, 'pm25': 11670}}, 'no': None, 'no2': None, 'o3': None, 'pm1': 2.12, 'pm10': 4.33, 'pm25': 2.46, 'raw_data_id': 62314430, 'rh': 39.2, 'sn': 'MOD-00480', 'temp': 25.9, 'timestamp': '2023-09-15T16:45:28', 'timestamp_local': '2023-0

## Download an event window for all devices

Next, we download data for the event window (Nov 7–10, 2024) for all devices visible to the API key. We store the results as raw files for later integration.


In [5]:
# import libraries
from pathlib import Path
import pandas as pd
from quantaq.utils import to_dataframe

# ---- Paths (match the PurpleAir acquisition notebook style) ----
BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw" / "quantaq"
RAW_DIR.mkdir(parents=True, exist_ok=True)

# ---- Event window ----
EVENT_START = "2024-11-07"
EVENT_END   = "2024-11-10"

# ---- Device inventory (metadata) ----
devices = client.devices.list()

devices_df = pd.DataFrame(devices)
devices_meta_path = RAW_DIR / "SouthBronxQuantAQDevices.parquet"
devices_df.to_parquet(devices_meta_path, index=False)
print(f"Saved device metadata to {devices_meta_path}")

# ---- Download measurements ----
rows = []
for device in devices:
    sn = device.get("sn")
    desc = device.get("description", "")
    if not sn:
        continue

    print(f"Downloading: {sn} - {desc}")

    # For each device, query the API for each day in the event window
    for each in pd.date_range(start=EVENT_START, end=EVENT_END, freq="D"):
        day_df = to_dataframe(client.data.bydate(sn=sn, date=str(each.date())))
        if day_df is not None and len(day_df) > 0:
            # Keep identifiers for integration later
            day_df["sn"] = sn
            day_df["description"] = desc
            rows.append(day_df)

if len(rows) == 0:
    raise RuntimeError("No QuantAQ data returned for the requested event window.")

df = pd.concat(rows, ignore_index=True)

# Drop some fields that are not useful for this lesson (keep errors='ignore' for robustness)
fields_to_drop = [
    "met.rh", "met.temp", "met.wd", "met.ws", "met.ws_scalar",
    "model.gas.co", "model.gas.no", "model.gas.no2", "model.gas.o3",
    "model.pm.pm1", "model.pm.pm10", "model.pm.pm25",
]
df = df.drop(columns=fields_to_drop, errors="ignore")

# ---- Save outputs ----
out_parquet = RAW_DIR / "SouthBronx_QuantAQ_2024_11_07_to_11_10.parquet"
df.to_parquet(out_parquet, index=False)
print(f"Saved merged QuantAQ data to {out_parquet} with shape {df.shape}")

# Optional CSV for quick inspection / compatibility
out_csv = RAW_DIR / "SouthBronx_QuantAQ_2024_11_07_to_11_10.csv"
df.to_csv(out_csv, index=False)
print(f"Saved CSV copy to {out_csv}")


Saved device metadata to data\raw\quantaq\SouthBronxQuantAQDevices.parquet
Downloading: MOD-PM-01144 - MOD-PM-01144 - La Finca del Sur (110 E 138th St)
Downloading: MOD-PM-01145 - MOD-PM-01145 - St Ann's Episcopal Church
Downloading: MOD-PM-01146 - MOD-PM-01146 - The Family School
Downloading: MOD-PM-01147 - MOD-PM-01147 - 335 E 140th St
Downloading: MOD-PM-01148 - MOD-PM-01148 - Ethical Culture Fieldston School 
Downloading: MOD-PM-01149 - MOD-PM-01149, Association for Energy Affordability 
Downloading: MOD-PM-01150 - MOD-PM-01150 - Brook Park
Downloading: MOD-00478 - MOD-00478, SBU Office
Downloading: MOD-00479 - MOD-00479 - Bruckner Garden
Downloading: MOD-00480 - MOD-00480 - Harlem River Yards Waste Transfer Station
Downloading: MOD-00481 - MOD-00481 - South Bronx Unite Office (Outside)
Downloading: MOD-00482 - MOD-00482 - Governors Island (Liberty Vista)
Downloading: MOD-PM-01151 - MOD-PM-01151 - 408 E 136th St
Downloading: MOD-PM-01152 - MOD-PM-01152 - MSHS 223 (423 Willis Ave)
D

C:\Users\kyttm\AppData\Local\Temp\ipykernel_38184\1999844940.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(rows, ignore_index=True)


Saved merged QuantAQ data to data\raw\quantaq\SouthBronx_QuantAQ_2024_11_07_to_11_10.parquet with shape (232518, 20)
Saved CSV copy to data\raw\quantaq\SouthBronx_QuantAQ_2024_11_07_to_11_10.csv
